### Вариант 1
#### Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайтов Superjob и HH. Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
- Наименование вакансии.
- Предлагаемую зарплату (отдельно минимальную и максимальную).
- Ссылку на саму вакансию.
- Сайт, откуда собрана вакансия. ### По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [81]:
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
import re
import pandas as pd
import json
from selenium import webdriver
import time

In [82]:
# Определяем параметры

vac = input("Какую должность ищем: ")
max_list_sj = int(input("Сколько страниц Superjobs анализировать: "))
max_list_hh = int(input("Сколько страниц HH анализировать: "))

Какую должность ищем: Python
Сколько страниц Superjobs анализировать: 1
Сколько страниц HH анализировать: 1


In [83]:
#Идея привести все ЗП к одному виду мне понравилась, попробовал тоже реализовать это.
cur_req = requests.get("https://www.cbr-xml-daily.ru/daily_json.js")    
cur = json.loads(cur_req.text)    

In [84]:
# Забираем инфу с HH.ru


job_info = []
n = 0

for x in range(max_list_hh):
    #Т.к. hh не отдает все карточки а только 20, пришлось использовать silenium чтобы сначала расскрывать страничку, потом парсить.
    driver = webdriver.Chrome(executable_path=r"C:\Users\user\Git\Parcing\chromedriver.exe")
    driver.get(f'https://hh.ru/search/vacancy?area=1&fromSearchLine=true&text={vac}&from=suggest_post&page={x}')
    time.sleep(5)
    html = driver.page_source
    #resphh = requests.get(f'https://hh.ru/search/vacancy?area=1&fromSearchLine=true&text={vac}&from=suggest_post&page={x}', headers = {'User-agent':'Chrome/63.0.3239.108'}).text
    soup = bs(html, 'html.parser')
    cards = soup.find_all(name='div', attrs={"class" : "vacancy-serp-item"} )
    for i in cards:
        name_job = i.find('a', attrs = {'data-qa': "vacancy-serp__vacancy-title"}).text
        
        link_job = i.find('a', attrs = {'data-qa': "vacancy-serp__vacancy-title"}).get('href')
        
        employer = i.find('a', attrs = {'data-qa': "vacancy-serp__vacancy-employer"}).text
        employer_link = f"https://hh.ru{i.find('a', attrs = {'data-qa': 'vacancy-serp__vacancy-employer'}).get('href')}"
        
        address = i.find('div', attrs = {'data-qa': "vacancy-serp__vacancy-address"}).text
        
        cost = i.find('span', attrs = {'data-qa' : "vacancy-serp__vacancy-compensation"})
        min_cost = np.nan
        max_cost = np.nan
        if cost:
            cost = cost.text 
            cost = cost.replace("\u202f", "").replace(" ", "")
            if re.findall('^от[0-9]+', cost):
                min_cost = int(re.findall('[0-9]+', cost)[0])
            elif re.findall('^до[0-9]+', cost):
                max_cost = int(re.findall('[0-9]+', cost)[0])
            else:
                min_cost = int(re.findall('[0-9]+', cost)[0])
                max_cost = int(re.findall('[0-9]+', cost)[1])
            if not re.findall('руб.', cost):
                currency = re.findall('[A-Z]{2,5}', cost)[0]
                if not np.isnan(min_cost):
                    min_cost = int(min_cost*cur["Valute"][currency]["Value"])//1000*1000
                if not np.isnan(max_cost):
                    max_cost = int(max_cost*cur["Valute"][currency]["Value"])//1000*1000
        
        job_from = "hh.ru"
        job_info.append([])
        job_info[n].append(name_job)
        job_info[n].append(link_job)
        job_info[n].append(min_cost)
        job_info[n].append(max_cost)
        job_info[n].append(job_from)
        job_info[n].append(employer)
        job_info[n].append(employer_link)
        job_info[n].append(address)
        
        
        n+=1    

print(len(job_info))

<ipython-input-84-0fd89d4168d2>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:\Users\user\Git\Parcing\chromedriver.exe")


50


In [85]:
# Соберем данные в датафрейм

columns = ["Name job", "Link", "Min_salary", "Max_salary", "Where", "Employer","Employer_link", "Address"]
df = pd.DataFrame(job_info, columns=columns)

In [86]:
# Забираем инфу с Superjobs.ru


job_info2 = []
n = 0

for x in range(max_list_sj):
    resphh = requests.get(f'https://www.superjob.ru/vacancy/search/?keywords={vac}&geo%5Bt%5D%5B0%5D=4&page={x}', headers = {'User-agent':'Mozilla/5.0'}).text
    soup = bs(resphh, 'html.parser')
    cards = soup.find_all(name='div', attrs={"class" : "f-test-vacancy-item"})
    
    for i in cards:
        name_job = i.find('a', attrs = {'class': "_6AfZ9"}).text
        
        
        link_job = f"https://www.superjob.ru{i.find('a', attrs = {'class': '_6AfZ9'}).get('href')}"
        employer = i.find('a', attrs = {'class': "_205Zx"})
        if employer:
            employer = i.find('a', attrs = {'class': "_205Zx"}).text
        
            employer_link = f"https://www.superjob.ru{i.find('a', attrs = {'class': '_205Zx'}).get('href')}"
        else:
            employer = np.nan
            employer_link = np.nan

        address = i.find('span', attrs = {'class': 'f-test-text-company-item-location'}).find(lambda tag: len(tag.attrs) == 0).text
             
        cost = i.find('span', attrs = {'class' : '_2Wp8I'})
         
        cost = cost.text
        cost = cost.replace("\xa0", "").replace(" ", "")
        if not re.findall('[0-9]+', cost):
            min_cost = np.nan
            max_cost = np.nan                   
        elif re.findall('^от[0-9]+', cost):
            min_cost = int(re.findall('[0-9]+', cost)[0])
        elif re.findall('^до[0-9]+', cost):
            max_cost = int(re.findall('[0-9]+', cost)[0])
        elif len(re.findall('[0-9]+', cost)) == 1:
            min_cost = int(re.findall('[0-9]+', cost)[0])
            max_cost = int(re.findall('[0-9]+', cost)[0])

        else:
            min_cost = int(re.findall('[0-9]+', cost)[0])
            max_cost = int(re.findall('[0-9]+', cost)[1])
        
        if not re.findall('руб.', cost) and re.findall('[0-9]+', cost):
            currency = re.findall('[A-Z]{2,5}', cost)[0]
            if not np.isnan(min_cost):
                min_cost = int(min_cost*cur["Valute"][currency]["Value"])//1000*1000
            if not np.isnan(max_cost):
                max_cost = int(max_cost*cur["Valute"][currency]["Value"])//1000*1000
        
        job_from = "superjob.ru/"
        job_info2.append([])
        job_info2[n].append(name_job)
        job_info2[n].append(link_job)
        job_info2[n].append(min_cost)
        job_info2[n].append(max_cost)
        job_info2[n].append(job_from)
        job_info2[n].append(employer)
        job_info2[n].append(employer_link)
        job_info2[n].append(address)
        
        
        n+=1    

    

In [87]:
# Соберем данные в датафрейм

columns = ["Name job", "Link", "Min_salary", "Max_salary", "Where", "Employer", "Employer_link", "Address"]
df2 = pd.DataFrame(job_info2, columns=columns)

In [88]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Name job       20 non-null     object 
 1   Link           20 non-null     object 
 2   Min_salary     13 non-null     float64
 3   Max_salary     11 non-null     float64
 4   Where          20 non-null     object 
 5   Employer       20 non-null     object 
 6   Employer_link  20 non-null     object 
 7   Address        20 non-null     object 
dtypes: float64(2), object(6)
memory usage: 1.4+ KB


In [89]:
# Склеим 2 датафрейма

final_df = pd.concat([df,df2],ignore_index=True)
final_df.info()
final_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Name job       70 non-null     object 
 1   Link           70 non-null     object 
 2   Min_salary     54 non-null     float64
 3   Max_salary     43 non-null     float64
 4   Where          70 non-null     object 
 5   Employer       70 non-null     object 
 6   Employer_link  70 non-null     object 
 7   Address        70 non-null     object 
dtypes: float64(2), object(6)
memory usage: 4.5+ KB


,Name job,Link,Min_salary,Max_salary,Where,Employer,Employer_link,Address
0,Middle/Senior Python developer,https://hh.ru/vacancy/49194069?from=vacancy_se...,246000.0,NaN,hh.ru,Unikoom,https://hh.ru/employer/5144162,Москва
1,Python Developer (Data Science),https://hh.ru/vacancy/49232800?from=vacancy_se...,NaN,NaN,hh.ru,ООО IHS Global,https://hh.ru/employer/841087,Москва
2,Junior Backend Developer (Python / Django),https://hh.ru/vacancy/49222903?from=vacancy_se...,75000.0,NaN,hh.ru,BAWAGA,https://hh.ru/employer/4866194,"Москва, Нижегородская и еще 1"
3,Python разработчик,https://hh.ru/vacancy/47682360?from=vacancy_se...,NaN,380000.0,hh.ru,Точка,https://hh.ru/employer/2324020,Москва
4,Junior разработчик Python Django,https://hh.ru/vacancy/48768385?from=vacancy_se...,40000.0,80000.0,hh.ru,ООО Инсейлс Рус,https://hh.ru/employer/1805574,"Москва, Комсомольская"


In [90]:
final_df.describe()

,Min_salary,Max_salary
count,54.000000,43.000000
mean,167555.555556,240581.395349
std,82566.763533,105363.369563
min,15000.000000,60000.000000
25%,112500.000000,165000.000000
50%,150000.000000,230000.000000
75%,240000.000000,300000.000000
max,350000.000000,520000.000000


In [91]:
# Сохраним в csv формат.

final_df.to_csv('task1.csv')